# "Hello World" Example Custom Sklearn Model - Scoring Only
This notebook assumes the `toy_model_training.ipynb` notebook generated the custom model and saved it in onnx format.

In [1]:
import numpy as np
import pandas as pd
from io import StringIO

import onnxruntime as rt

## Generate data

In [2]:
test_data = """x1,x2,y
1,2,11
-3,1,-1
0,0,1
1,1,7
2,2,13
-1,-1,-5
10,10,61
"""
test_df = pd.read_csv(StringIO(test_data))
test_df

,x1,x2,y
0,1,2,11
1,-3,1,-1
2,0,0,1
3,1,1,7
4,2,2,13
5,-1,-1,-5
6,10,10,61


## restore model

In [3]:
# retrieve model from disk
sess = rt.InferenceSession('./toy_model.onnx')

# get model names
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name

## Score test data set

In [4]:
# score the data set
# generate predictions
list_of_predictions = sess.run(
    [label_name], 
    {input_name: test_df.drop(['y'], axis=1).to_numpy().astype(np.float32)}
)
prediction = list_of_predictions[0]

In [5]:
print(f'predictions:\n{prediction}')

predictions:
[[11.]
 [-1.]
 [ 1.]
 [ 7.]
 [13.]
 [-5.]
 [61.]]


## Explore the model metadata, inputs and outputs

In [6]:
model_metadata = sess.get_modelmeta()
print(f'description: {model_metadata.description}')
print(f'domain: {model_metadata.domain}')
print(f'graph_name: {model_metadata.graph_name}')
print(f'producer_name: {model_metadata.producer_name}')
print(f'version: {model_metadata.version}')

description: 
domain: ai.onnx
graph_name: 2f407863ce6f4776bbd0fca9c151c614
producer_name: skl2onnx
version: 0


In [7]:
print("model inputs:")
for i in sess.get_inputs():
    print(f'name: {i.name}, shape:{i.shape}, dtype:{i.type}')

model inputs:
name: float_input, shape:[None, 2], dtype:tensor(float)


In [8]:
print("model outputs:")
for o in sess.get_outputs():
    print(f'name: {o.name}, shape: {o.shape}, dtype: {o.type}')

model outputs:
name: variable, shape: [None, 1], dtype: tensor(float)
